In [8]:
from collections import Counter
import numpy as np
import pandas as pd
import pickle

In [5]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col = 0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
    df.fillna(0, inplace = True)
    return tickers, df

#process_data_for_labels('XOM')

In [11]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement: #buy 
            return 1
        if col < -requirement: # sell
            return -1
    return 0

In [12]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)
    
    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, 
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread: ', Counter(str_vals))
    
    df.fillna(0, inplace = True)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace = True)
    
    df_vals = df[[ticker for ticker in tickers]].pct_change()  #normalizing the values
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace= True)
    
    X= df_vals.values
    y = df['{}_target'.format(ticker)].values
    
    return X, y, df

In [13]:
extract_featuresets('XOM')

Data spread:  Counter({'1': 1435, '-1': 1265, '0': 1143})


(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.00813105, -0.01007022,  0.        , ..., -0.00037824,
         -0.01365146,  0.        ],
        [-0.01223558, -0.011472  ,  0.        , ..., -0.00781297,
         -0.00361068,  0.        ],
        ...,
        [ 0.05164814,  0.04128391,  0.03216574, ...,  0.06444952,
          0.09116694,  0.10627345],
        [ 0.02771862, -0.00967005, -0.00448969, ...,  0.04053503,
          0.04121799, -0.0053632 ],
        [ 0.03035961,  0.03686069,  0.04204799, ...,  0.09194382,
          0.04992866,  0.00903972]]),
 array([0, 0, 1, ..., 1, 1, 0]),
                    MMM        ABT       ABBV        ABMD         ACN  \
 Date                                                                   
 2005-01-03   55.372093  11.840737   0.000000   15.010000   19.824249   
 2005-01-04   54.921860  11.721498   0.000000   14.980000   19.358147   
 2005-01-05   54.249859  11.587029   0.000000   